# **k8o5/studio**

In [ ]:
!pip install -q yt_dlp

import os
import json
import base64
import subprocess
import sys
from IPython.display import display, HTML
from google.colab import output
import yt_dlp

# Ensure yt_dlp is available
try:
    import yt_dlp
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "yt_dlp"])
    import yt_dlp

def get_file_list():
    return sorted([f for f in os.listdir('.') if f.endswith('.mp3')])

def fetch_song_handler(fname):
    try:
        with open(fname, "rb") as f:
            enc = base64.b64encode(f.read()).decode('utf-8')
            data_str = f"data:audio/mpeg;base64,{enc}"
            # Safe filename escaping for JS
            output.eval_js(f"receiveSongData({json.dumps(fname)}, {json.dumps(data_str)})")
    except Exception as e:
        print(f"Error fetching {fname}: {e}")
        output.eval_js(f"showStatus('Error loading file', false)")

def dl_handler(query):
    output.eval_js("showStatus('Processing Master...', true)")
    opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{'key': 'FFmpegExtractAudio','preferredcodec': 'mp3','preferredquality': '320'}],
        'outtmpl': '%(title)s.%(ext)s',
        'quiet': True,
        'noplaylist': True,
        'extractor_args': {'youtube': {'player_client': ['ios', 'android', 'web']}}
    }

    if query.startswith("http"):
        search_targets = [query]
    else:
        # Fallback Strategy: SoundCloud -> YouTube
        search_targets = [f"scsearch1:{query}", f"ytsearch1:{query}"]

    success = False
    for target in search_targets:
        try:
            with yt_dlp.YoutubeDL(opts) as ydl:
                info = ydl.extract_info(target, download=True)
                if 'soundcloud' in target and info.get('duration', 0) < 60:
                    raise Exception("Snippet")
                success = True
                break
        except: continue

    if success:
        f = get_file_list()
        output.eval_js(f'updateLib({json.dumps(f)})')
        output.eval_js("showStatus('Download Complete', false)")
    else:
        output.eval_js(f"showStatus('Source Error', false)")

def del_handler(fname):
    if os.path.exists(fname):
        os.remove(fname)
        f = get_file_list()
        output.eval_js(f'updateLib({json.dumps(f)})')

output.register_callback('k8o5.dl', dl_handler)
output.register_callback('k8o5.del', del_handler)
output.register_callback('k8o5.fetch', fetch_song_handler)

files = get_file_list()

HTML_CODE = f"""
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<title>k8o5/studio - AAA Overdrive</title>
<style>
@import url('https://fonts.googleapis.com/css2?family=Plus+Jakarta+Sans:wght@400;600;800&display=swap');

:root {{
    --bg: #050505;
    --surf: #0c0c0c;
    --border: rgba(255,255,255,0.08);
    --text: #ffffff;
    --acc: #1DB954;
    --acc-hot: #ff3b3b;
    --glass: rgba(255,255,255,0.03);
}}

* {{ box-sizing: border-box; margin: 0; padding: 0; outline: none; user-select: none; -webkit-user-drag: none; }}

body {{
    background: #000; color: var(--text); font-family: 'Plus Jakarta Sans', sans-serif;
    height: 100vh; width: 100vw; display: flex; align-items: center; justify-content: center; overflow: hidden;
}}

#app {{
    width: 100%; height: 100%; max-width: 1200px; max-height: 850px;
    background: var(--bg); display: grid;
    grid-template-rows: 75px 1fr 110px; grid-template-columns: 240px 1fr;
    grid-template-areas: "head head" "side main" "play play";
    border: 1px solid var(--border); border-radius: 20px; box-shadow: 0 50px 100px rgba(0,0,0,0.8);
    position: relative; overflow: hidden;
}}

/* UI COMPONENTS */
.header {{
    grid-area: head; border-bottom: 1px solid var(--border);
    display: flex; align-items: center; justify-content: space-between;
    padding: 0 30px; background: rgba(5,5,5,0.8); backdrop-filter: blur(20px); z-index: 100;
}}
.header-search input {{
    width: 400px; background: var(--glass); border: 1px solid var(--border); color: #fff;
    padding: 10px 20px; border-radius: 10px; font-size: 13px;
}}
.status-bar {{
    background: var(--glass); padding: 8px 16px; border-radius: 20px; border: 1px solid var(--border);
    font-size: 10px; font-weight: 800; display: flex; align-items: center; gap: 10px;
    opacity: 0; transition: 0.5s; text-transform: uppercase; letter-spacing: 1px;
}}
.status-bar.show {{ opacity: 1; }}

.sidebar {{
    grid-area: side; background: var(--surf); border-right: 1px solid var(--border);
    padding: 30px 15px; display: flex; flex-direction: column; gap: 5px;
}}
.nav-btn {{
    padding: 14px 20px; cursor: pointer; color: #666; border-radius: 10px;
    font-size: 14px; font-weight: 600; transition: 0.3s;
}}
.nav-btn:hover {{ color: #fff; background: var(--glass); }}
.nav-btn.active {{ color: var(--acc); background: rgba(29,185,84,0.1); }}

.main {{ grid-area: main; position: relative; background: #080808; overflow: hidden; }}
.page {{ display: none; height: 100%; overflow-y: auto; padding: 40px; z-index: 2; position: relative; }}
.page.active {{ display: block; }}

.grid {{ display: grid; grid-template-columns: repeat(auto-fill, minmax(170px, 1fr)); gap: 25px; }}
.card {{ cursor: pointer; position: relative; }}
.art-box {{
    width: 100%; aspect-ratio: 1; border-radius: 15px; background: #111;
    position: relative; overflow: hidden; margin-bottom: 12px;
    box-shadow: 0 10px 20px rgba(0,0,0,0.4); transition: 0.4s cubic-bezier(0.2, 1, 0.2, 1);
}}
.card:hover .art-box {{ transform: translateY(-5px); box-shadow: 0 15px 30px rgba(0,0,0,0.6); }}
.song-title {{ font-size: 14px; font-weight: 700; color: #eee; white-space: nowrap; overflow: hidden; text-overflow: ellipsis; }}
.del-btn {{
    position: absolute; top: 10px; right: 10px; width: 28px; height: 28px;
    background: rgba(0,0,0,0.7); color: #fff; border-radius: 50%;
    display: flex; align-items: center; justify-content: center; opacity: 0; transition: 0.2s;
}}
.card:hover .del-btn {{ opacity: 1; }}

/* FX RACK STYLE (RESTORED) */
.fx-grid {{ display: grid; grid-template-columns: repeat(auto-fit, minmax(250px, 1fr)); gap: 20px; }}
.fx-card {{ background: var(--surf); border: 1px solid var(--border); padding: 25px; border-radius: 15px; }}
.fx-title {{ font-size: 11px; font-weight: 800; color: var(--acc); letter-spacing: 2px; margin-bottom: 20px; }}
.range {{ -webkit-appearance: none; width: 100%; height: 4px; background: #222; border-radius: 2px; outline: none; }}
.range::-webkit-slider-thumb {{ -webkit-appearance: none; width: 14px; height: 14px; background: #fff; border-radius: 50%; border: 2px solid var(--acc); }}

/* PLAYER BAR */
.player {{
    grid-area: play; background: #000; border-top: 1px solid var(--border);
    display: grid; grid-template-columns: 280px 1fr 280px; align-items: center; padding: 0 40px; z-index: 100;
}}
.prog-bar {{ flex: 1; height: 5px; background: #111; border-radius: 10px; cursor: pointer; position: relative; overflow: hidden; }}
.prog-fill {{ width: 0%; height: 100%; background: var(--acc); }}

.vol-wrap {{ display: flex; align-items: center; justify-content: flex-end; gap: 12px; }}
.vol-track {{ -webkit-appearance: none; width: 100px; height: 4px; background: #222; border-radius: 2px; }}
.vol-val {{ font-size: 11px; font-weight: 800; width: 40px; color: #555; text-align: right; }}
.vol-val.boost {{ color: var(--acc-hot); animation: blink 0.6s infinite; }}

@keyframes blink {{ 50% {{ opacity: 0.6; }} }}
#viz {{ position: absolute; top: 0; left: 0; width: 100%; height: 100%; pointer-events: none; opacity: 0.4; z-index: 1; mix-blend-mode: screen; }}

.spinner {{ width: 14px; height: 14px; border: 2px solid rgba(255,255,255,0.1); border-top: 2px solid var(--acc); border-radius: 50%; animation: spin 1s linear infinite; }}
@keyframes spin {{ to {{ transform: rotate(360deg); }} }}
</style>
</head>
<body>

<div id="app">
    <div class="header">
        <div style="font-weight:800; font-size:18px">k8o5<span style="color:var(--acc);font-weight:400">/STUDIO</span></div>
        <div class="header-search">
            <input type="text" id="lib-search" placeholder="Search or paste link..." oninput="filterLib()" onkeydown="handleSearch(event)">
        </div>
        <div class="status-bar" id="status"><div class="spinner"></div><span id="stat-txt">Ready</span></div>
    </div>

    <div class="sidebar">
        <div class="nav-btn active" onclick="nav('lib', this)">Library</div>
        <div class="nav-btn" onclick="nav('fx', this)">FX Studio</div>
    </div>

    <div class="main">
        <canvas id="viz"></canvas>
        <div id="p-lib" class="page active">
            <div class="grid" id="grid"></div>
        </div>
        <div id="p-fx" class="page">
            <h1>Mastering Rack</h1>
            <div class="fx-grid">
                <div class="fx-card">
                    <div class="fx-title">ENGINE</div>
                    <div style="font-size:12px; color:#666; margin-bottom:10px">Playback Speed: <span id="v-spd">1.00x</span></div>
                    <input type="range" class="range" id="sl-spd" min="0.25" max="2.0" step="0.01" value="1">
                </div>
                <div class="fx-card">
                    <div class="fx-title">TONE</div>
                    <div style="margin-bottom:15px">
                        <div style="font-size:12px; color:#666; margin-bottom:10px">Lows: <span id="v-low">0dB</span></div>
                        <input type="range" class="range" id="sl-low" min="-24" max="24" step="0.1" value="0">
                    </div>
                    <div>
                        <div style="font-size:12px; color:#666; margin-bottom:10px">Highs: <span id="v-hi">0dB</span></div>
                        <input type="range" class="range" id="sl-hi" min="-24" max="24" step="0.1" value="0">
                    </div>
                </div>
                <div class="fx-card">
                    <div class="fx-title">SPACE</div>
                    <div style="font-size:12px; color:#666; margin-bottom:10px">Reverb Mix: <span id="v-rev">0%</span></div>
                    <input type="range" class="range" id="sl-rev" min="0" max="0.8" step="0.01" value="0">
                </div>
            </div>
        </div>
    </div>

    <div class="player">
        <div style="display:flex; align-items:center; gap:15px">
            <div id="art" style="width:50px; height:50px; border-radius:8px; background:#111; overflow:hidden"></div>
            <div style="overflow:hidden">
                <div id="t-name" style="font-size:14px; font-weight:700; white-space:nowrap; overflow:hidden; text-overflow:ellipsis; width:180px">No Track</div>
                <div id="t-stat" style="font-size:10px; color:var(--acc); font-weight:800; text-transform:uppercase">Idle</div>
            </div>
        </div>
        <div style="display:flex; flex-direction:column; align-items:center; width:100%">
            <div style="display:flex; align-items:center; gap:30px; margin-bottom:10px">
                <button style="background:none; border:none; color:#555; cursor:pointer" onclick="prev()">⏮</button>
                <button id="bp" style="width:42px; height:42px; background:#fff; border-radius:50%; border:none; cursor:pointer" onclick="toggle()">▶</button>
                <button style="background:none; border:none; color:#555; cursor:pointer" onclick="next(true)">⏭</button>
            </div>
            <div style="display:flex; align-items:center; gap:10px; width:100%; max-width:500px">
                <span id="curr-time" style="font-size:10px; color:#444">0:00</span>
                <div class="prog-bar" id="prog"><div class="prog-fill" id="pfill"></div></div>
                <span id="total-time" style="font-size:10px; color:#444">0:00</span>
            </div>
        </div>
        <div class="vol-wrap">
            <input type="range" class="vol-track" id="sl-vol" min="0" max="2" step="0.01" value="0.8">
            <div class="vol-val" id="v-vol">80%</div>
        </div>
    </div>
</div>

<script>
    let files = {json.dumps(files)};
    let audioCache = {{}};
    let idx=-1, playing=false, ac, src, gain, anal, lim, buf, st=0, ot=0, dur=0;
    let low, high, verb, dry, wet;

    const $ = id => document.getElementById(id);

    function init() {{
        if(ac) return;
        ac = new (window.AudioContext || window.webkitAudioContext)();
        gain = ac.createGain();
        anal = ac.createAnalyser(); anal.fftSize = 512;

        // HOKUS POKUS: Master Limiter (prevents 200% volume from shattering)
        lim = ac.createDynamicsCompressor();
        lim.threshold.value = -1.0; lim.knee.value = 0; lim.ratio.value = 20; lim.attack.value = 0.005; lim.release.value = 0.1;

        low = ac.createBiquadFilter(); low.type="lowshelf"; low.frequency.value=350;
        high = ac.createBiquadFilter(); high.type="highshelf"; high.frequency.value=3200;

        verb = ac.createConvolver(); dry = ac.createGain(); wet = ac.createGain();
        let len = ac.sampleRate * 2;
        let imp = ac.createBuffer(2, len, ac.sampleRate);
        for(let i=0; i<2; i++) {{
            let ch = imp.getChannelData(i);
            for(let j=0; j<len; j++) ch[j] = (Math.random()*2-1)*Math.pow(1-j/len, 1.5);
        }}
        verb.buffer = imp;

        // Routing
        gain.connect(lim);
        lim.connect(anal);
        anal.connect(ac.destination);

        applyFx();
        vizLoop();
    }}

    function handleSearch(e) {{ if(e.key==='Enter' && $('lib-search').value.trim()) {{ google.colab.kernel.invokeFunction('k8o5.dl', [$('lib-search').value]); $('lib-search').value=''; }} }}

    async function load(i) {{
        init(); if(playing) stop(true); idx = i;
        let n = files[i]; $('t-name').innerText = n.replace('.mp3',''); $('t-stat').innerText = "Buffering...";
        if(audioCache[n]) processAudio(audioCache[n]);
        else google.colab.kernel.invokeFunction('k8o5.fetch', [n]);
    }}

    window.receiveSongData = (fname, base64Str) => {{ audioCache[fname] = base64Str; if(files[idx] === fname) processAudio(base64Str); }};

    async function processAudio(base64Str) {{
        try {{
            let b = Uint8Array.from(atob(base64Str.split(',')[1]), c => c.charCodeAt(0)).buffer;
            buf = await ac.decodeAudioData(b); dur = buf.duration; $('total-time').innerText = fmtTime(dur); play(0);
        }} catch(e) {{ $('t-stat').innerText = "Error"; }}
    }}

    function play(off) {{
        src = ac.createBufferSource(); src.buffer = buf;
        src.connect(low); low.connect(high); high.connect(dry); high.connect(wet);
        dry.connect(gain); wet.connect(verb); verb.connect(gain);
        applyFx(); src.start(0, off);
        st = ac.currentTime - off; playing = true;
        $('bp').innerText = "⏸"; $('t-stat').innerText = "Streaming";
        src.onended = () => {{ if(playing && (ac.currentTime-st)>=dur-0.2) next(false); }};
    }}

    function stop(mem) {{
        if(src) {{ src.stop(); src.disconnect(); }}
        if(playing) ot = ac.currentTime - st; playing = false; if(!mem) ot = 0;
        $('bp').innerText = "▶"; $('t-stat').innerText = "Paused";
    }}

    function toggle() {{ if(idx<0 && files.length) load(0); else if(playing) stop(true); else if(buf) play(ot); }}
    function next(m) {{ if(files.length) load((idx+1)%files.length); }}
    function prev() {{ if(files.length) load((idx-1+files.length)%files.length); }}

    function applyFx() {{
        let spd = parseFloat($('sl-spd').value); $('v-spd').innerText = spd.toFixed(2) + 'x';
        let l = parseFloat($('sl-low').value); $('v-low').innerText = l + 'dB';
        let h = parseFloat($('sl-hi').value); $('v-hi').innerText = h + 'dB';
        let rv = parseFloat($('sl-rev').value); $('v-rev').innerText = Math.round(rv*100) + '%';
        let vol = parseFloat($('sl-vol').value);
        $('v-vol').innerText = Math.round(vol*100) + '%';
        $('v-vol').className = vol > 1 ? 'vol-val boost' : 'vol-val';

        if(!ac) return;
        if(src) src.playbackRate.value = spd;
        low.gain.value = l; high.gain.value = h;
        dry.gain.value = 1 - rv; wet.gain.value = rv;
        gain.gain.value = vol;
    }}

    ['sl-spd','sl-low','sl-hi','sl-rev','sl-vol'].forEach(id => $(id).addEventListener('input', applyFx));

    $('prog').addEventListener('click', e => {{
        if(!buf) return; let r = $('prog').getBoundingClientRect();
        stop(true); play(dur * ((e.clientX - r.left)/r.width));
    }});

    function fmtTime(s) {{ let m=Math.floor(s/60), sec=Math.floor(s%60); return m+":"+(sec<10?"0":"")+sec; }}

    setInterval(() => {{
        if(playing) {{
            let cur = ac.currentTime - st;
            $('pfill').style.width = (cur/dur)*100 + '%';
            $('curr-time').innerText = fmtTime(cur);
        }}
    }}, 100);

    // DYNAMIC AAA VISUALIZER
    const cv = $('viz'), cx = cv.getContext('2d');
    const res = () => {{ cv.width = $('app').clientWidth; cv.height = $('app').clientHeight; }};
    window.onresize = res; setTimeout(res, 500);

    function vizLoop() {{
        requestAnimationFrame(vizLoop);
        if(!anal || !playing) {{ cx.clearRect(0,0,cv.width,cv.height); return; }}
        let d = new Uint8Array(anal.frequencyBinCount);
        anal.getByteFrequencyData(d);
        cx.clearRect(0,0,cv.width,cv.height);
        let bw = (cv.width/d.length) * 2.5, x=0;
        for(let i=0; i<d.length; i++) {{
            let h = (d[i]/255) * (cv.height * 0.6); // Scales to window height
            cx.fillStyle = i % 2 === 0 ? 'rgba(29, 185, 84, 0.4)' : 'rgba(255, 255, 255, 0.1)';
            cx.fillRect(x, cv.height-h, bw-1, h);
            x += bw;
        }}
    }}

    function render() {{
        let h = '';
        files.forEach((f, i) => {{
            let c = (f.charCodeAt(0)*20)%360;
            h+=`<div class="card" onclick="load(${{i}})">
                <div class="art-box">
                    <div style="width:100%;height:100%;background:linear-gradient(45deg, hsl(${{c}},40%,20%), #000)"></div>
                    <div class="del-btn" onclick="del(${{JSON.stringify(f)}}, event)">✕</div>
                </div>
                <div class="song-title">${{f.replace('.mp3','')}}</div>
            </div>`;
        }});
        $('grid').innerHTML = h || '<div style="grid-column:1/-1;text-align:center;padding:100px;opacity:0.2">STUDIO EMPTY</div>';
    }}

    window.nav = (p, el) => {{
        document.querySelectorAll('.page').forEach(x=>x.classList.remove('active')); $('p-'+p).classList.add('active');
        document.querySelectorAll('.nav-btn').forEach(x=>x.classList.remove('active')); el.classList.add('active');
    }};

    window.filterLib = () => {{
        let q = $('lib-search').value.toLowerCase();
        document.querySelectorAll('.card').forEach(c => {{
            c.style.display = c.innerText.toLowerCase().includes(q) ? 'block' : 'none';
        }});
    }};

    window.del = (n, e) => {{ e.stopPropagation(); if(confirm('Delete?')) google.colab.kernel.invokeFunction('k8o5.del', [n]); }};
    window.updateLib = (f) => {{ files=f; render(); }};
    window.showStatus = (m, l) => {{
        $('status').classList.add('show'); $('stat-txt').innerText=m;
        document.querySelector('.spinner').style.display=l?'block':'none';
        if(!l) setTimeout(()=>$('status').classList.remove('show'),3000);
    }};

    render();
</script>
</body>
</html>
"""
display(HTML(HTML_CODE))